# Spark Coding Assignment by Shubham Pawar

In [1]:
import findspark ##importing findspark to connect jupyter with pyspark

findspark.init("C:\\Spark\\spark-2.4.7-bin-hadoop2.7")

In [2]:
import pyspark ##importing pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

df = spark.sql("select 'spark' as hello ")

df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [3]:
startup_df=spark.read.csv("C:/Users/Mehul/startup.csv",inferSchema=True,header=True)

In [4]:
consumerinter_df=spark.read.parquet("C:/Users/Shubham/consumerInternet.parquet")

In [5]:
df2=startup_df.unionAll(consumerinter_df)   ##Combining both

df2.printSchema()

root
 |-- Sr_No: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Startup_Name: string (nullable = true)
 |-- Industry_Vertical: string (nullable = true)
 |-- SubVertical: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Investors_Name: string (nullable = true)
 |-- InvestmentnType: string (nullable = true)
 |-- Amount_in_USD: string (nullable = true)
 |-- Remarks: string (nullable = true)



In [6]:
df2.show(2)

+-----+----------+------------+-----------------+--------------------+---------+--------------------+--------------------+-------------+-------+
|Sr_No|      Date|Startup_Name|Industry_Vertical|         SubVertical|     City|      Investors_Name|     InvestmentnType|Amount_in_USD|Remarks|
+-----+----------+------------+-----------------+--------------------+---------+--------------------+--------------------+-------------+-------+
|    1|09/01/2020|      BYJU’S|           E-Tech|          E-learning|Bengaluru|Tiger Global Mana...|Private Equity Round| 20,00,00,000|   null|
|    2|13/01/2020|      Shuttl|   Transportation|App based shuttle...|  Gurgaon|Susquehanna Growt...|            Series C|    80,48,394|   null|
+-----+----------+------------+-----------------+--------------------+---------+--------------------+--------------------+-------------+-------+
only showing top 2 rows



In [7]:
##How many startups are there in Pune City

In [8]:
df2.createOrReplaceTempView("df2")

StartupsPune = spark.sql("SELECT COUNT(Startup_Name)AS TotalStartupCount FROM df2 WHERE City = 'Pune'")
StartupsPune.show()

+-----------------+
|TotalStartupCount|
+-----------------+
|              105|
+-----------------+



In [9]:
###How many startups in Pune got their Seed/ Angel Funding?

In [10]:
FundedStartups = spark.sql("SELECT COUNT(Startup_Name) AS TotalStartupFunded FROM df2 WHERE InvestmentnType like 'Seed%/%' AND City = 'Pune'")
FundedStartups.show()

+------------------+
|TotalStartupFunded|
+------------------+
|                 6|
+------------------+



In [11]:
##What is the total amount raised by startups in Pune City? Hint - use regex_replace to get rid of null

In [12]:
PuneRaisedAmount = spark.sql("SELECT SUM(CAST(regexp_replace(regexp_replace(Amount_in_USD, 'N/A', '00'), ',', '') AS INT)) AmountRaised FROM df2 WHERE City = 'Pune'")
PuneRaisedAmount.show()

+------------+
|AmountRaised|
+------------+
|   633082000|
+------------+



In [13]:
##What are the top 5 Industry_Vertical which has the highest number of startups in India?

In [14]:
Top5 = spark.sql("SELECT Industry_Vertical, COUNT(Startup_Name) as StartupCount FROM df2 GROUP BY Industry_Vertical ORDER BY COUNT(Startup_Name) DESC LIMIT 5")
Top5.show()

+-----------------+------------+
|Industry_Vertical|StartupCount|
+-----------------+------------+
|Consumer Internet|         941|
|       Technology|         478|
|        eCommerce|         186|
|              nan|         171|
|       Healthcare|          70|
+-----------------+------------+



In [15]:
##Find the top Investor(by amount) of each year.

In [27]:
Inv = spark.sql("SELECT Investors_Name FROM df2 WHERE CAST(regexp_replace(regexp_replace(Amount_in_USD, 'N/A', '00'), ',', '') AS INT) = max(CAST(regexp_replace(regexp_replace(Amount_in_USD, 'N/A', '00'), ',', '') AS INT))")

In [28]:
Inv.show()

Py4JJavaError: An error occurred while calling o404.showString.
: java.lang.UnsupportedOperationException: Cannot evaluate expression: max(cast(regexp_replace(regexp_replace(input[1, string, true], N/A, 00), ,, ) as int))
	at org.apache.spark.sql.catalyst.expressions.Unevaluable$class.doGenCode(Expression.scala:261)
	at org.apache.spark.sql.catalyst.expressions.aggregate.AggregateExpression.doGenCode(interfaces.scala:87)
	at org.apache.spark.sql.catalyst.expressions.Expression$$anonfun$genCode$2.apply(Expression.scala:108)
	at org.apache.spark.sql.catalyst.expressions.Expression$$anonfun$genCode$2.apply(Expression.scala:105)
	at scala.Option.getOrElse(Option.scala:121)
	at org.apache.spark.sql.catalyst.expressions.Expression.genCode(Expression.scala:105)
	at org.apache.spark.sql.catalyst.expressions.BinaryExpression.nullSafeCodeGen(Expression.scala:526)
	at org.apache.spark.sql.catalyst.expressions.BinaryExpression.defineCodeGen(Expression.scala:508)
	at org.apache.spark.sql.c